In [78]:
import torch
from torch import Tensor
from torch import nn
from torch import optim
from torch.utils.data import Dataset, DataLoader

# 01分类

小于等于0为0,否则为1

# model

In [79]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [80]:
model = nn.Sequential(
    nn.Linear(1, 3),
    nn.ReLU(),
    nn.Linear(3, 1),
).to(device)
model

Sequential(
  (0): Linear(in_features=1, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)

In [81]:
optimizer = optim.SGD(model.parameters(), lr=0.1)
optimizer

SGD (
Parameter Group 0
    dampening: 0
    differentiable: False
    foreach: None
    lr: 0.1
    maximize: False
    momentum: 0
    nesterov: False
    weight_decay: 0
)

In [82]:
# 要求模型输出个数为1
loss_fn = nn.BCEWithLogitsLoss()
loss_fn

BCEWithLogitsLoss()

# data

In [83]:
train_x = torch.randn(10000, 1)
train_x

tensor([[ 0.2869],
        [-2.7048],
        [ 0.8248],
        ...,
        [ 0.4532],
        [ 0.2260],
        [-0.3508]])

In [84]:
train_y = (train_x > 0).type(torch.float)
train_y

tensor([[1.],
        [0.],
        [1.],
        ...,
        [1.],
        [1.],
        [0.]])

In [85]:
val_x = torch.rand([10000, 1]) * 2 - 1
val_x

tensor([[-0.4051],
        [ 0.0679],
        [-0.3509],
        ...,
        [ 0.0711],
        [-0.3738],
        [ 0.3602]])

In [86]:
val_y = (val_x > 0).type(torch.float)
val_y

tensor([[0.],
        [1.],
        [0.],
        ...,
        [1.],
        [0.],
        [1.]])

In [87]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __getitem__(self, index):
        return self.X[index], self.y[index]

    def __len__(self):
        return len(self.X)

In [88]:
train_dataloader = DataLoader(Data(train_x, train_y), batch_size=1000, shuffle=True)
train_dataloader

In [89]:
val_dataloader = DataLoader(Data(val_x, val_y), batch_size=1000, shuffle=True)
val_dataloader

# train

In [90]:
epochs = 100

In [91]:
for epoch in range(1, epochs+1):
    model.train()
    y_trues = []
    y_preds = []
    losses = []
    for x, y in train_dataloader:
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_pred: Tensor = model(x)
        loss: Tensor = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()

        y_trues.append(y)
        y_preds.append((y_pred.sigmoid() > 0.5).type(torch.float))    # 大于0.5代表正确
        losses.append(loss.item())

    acc = (torch.cat(y_trues) == torch.cat(y_preds)).type(torch.float).mean().item()
    loss_mean = torch.tensor(losses).mean().item()
    print(f"epoch: {epoch}, train, acc: {acc:.6f}, loss: {loss_mean:.6f}")

    model.eval()
    y_trues = []
    y_preds = []
    losses = []
    for x, y in val_dataloader:
        x, y = x.to(device), y.to(device)
        with torch.no_grad():
            y_pred: Tensor = model(x)
        loss: Tensor = loss_fn(y_pred, y)
        y_trues.append(y)
        y_preds.append((y_pred.sigmoid() > 0.5).type(torch.float))    # 大于0.5代表正确
        losses.append(loss.item())

    acc = (torch.cat(y_trues) == torch.cat(y_preds)).type(torch.float).mean().item()
    loss_mean = torch.tensor(losses).mean().item()
    print(f"epoch: {epoch}, val, acc: {acc:.6f}, loss: {loss_mean:.6f}")

epoch: 1, train, acc: 0.128900, loss: 0.761887
epoch: 1, val, acc: 0.506600, loss: 0.705111
epoch: 2, train, acc: 0.487000, loss: 0.687132


epoch: 2, val, acc: 0.506600, loss: 0.670082
epoch: 3, train, acc: 0.633300, loss: 0.637318
epoch: 3, val, acc: 0.703100, loss: 0.633289
epoch: 4, train, acc: 0.829200, loss: 0.582872
epoch: 4, val, acc: 0.838200, loss: 0.587674
epoch: 5, train, acc: 0.900900, loss: 0.520129
epoch: 5, val, acc: 0.911200, loss: 0.533740
epoch: 6, train, acc: 0.943200, loss: 0.453638
epoch: 6, val, acc: 0.951200, loss: 0.475961
epoch: 7, train, acc: 0.970300, loss: 0.390047
epoch: 7, val, acc: 0.971800, loss: 0.420134
epoch: 8, train, acc: 0.982000, loss: 0.334677
epoch: 8, val, acc: 0.983600, loss: 0.370869
epoch: 9, train, acc: 0.988800, loss: 0.289440
epoch: 9, val, acc: 0.990600, loss: 0.329393
epoch: 10, train, acc: 0.994100, loss: 0.253618
epoch: 10, val, acc: 0.993600, loss: 0.295208
epoch: 11, train, acc: 0.995100, loss: 0.225451
epoch: 11, val, acc: 0.995500, loss: 0.267243
epoch: 12, train, acc: 0.997300, loss: 0.203156
epoch: 12, val, acc: 0.996400, loss: 0.244267
epoch: 13, train, acc: 0.9977

# test

In [ ]:
with torch.inference_mode():
    x = torch.tensor([-0.01, -0.001, 0.001, 0.01]).reshape(-1, 1).to(device)
    print(model(x).cpu().sigmoid())

tensor([[0.4564],
        [0.4909],
        [0.4987],
        [0.5333]])


In [92]:
test_x = torch.logspace(-4, 0, 100)
test_x

tensor([1.0000e-04, 1.0975e-04, 1.2045e-04, 1.3219e-04, 1.4508e-04, 1.5923e-04,
        1.7475e-04, 1.9179e-04, 2.1049e-04, 2.3101e-04, 2.5354e-04, 2.7826e-04,
        3.0539e-04, 3.3516e-04, 3.6784e-04, 4.0370e-04, 4.4306e-04, 4.8626e-04,
        5.3367e-04, 5.8570e-04, 6.4281e-04, 7.0548e-04, 7.7426e-04, 8.4975e-04,
        9.3260e-04, 1.0235e-03, 1.1233e-03, 1.2328e-03, 1.3530e-03, 1.4850e-03,
        1.6298e-03, 1.7886e-03, 1.9630e-03, 2.1544e-03, 2.3645e-03, 2.5950e-03,
        2.8480e-03, 3.1257e-03, 3.4305e-03, 3.7649e-03, 4.1320e-03, 4.5349e-03,
        4.9770e-03, 5.4623e-03, 5.9948e-03, 6.5793e-03, 7.2208e-03, 7.9248e-03,
        8.6975e-03, 9.5455e-03, 1.0476e-02, 1.1498e-02, 1.2619e-02, 1.3849e-02,
        1.5199e-02, 1.6681e-02, 1.8307e-02, 2.0092e-02, 2.2051e-02, 2.4201e-02,
        2.6561e-02, 2.9151e-02, 3.1993e-02, 3.5112e-02, 3.8535e-02, 4.2292e-02,
        4.6416e-02, 5.0941e-02, 5.5908e-02, 6.1359e-02, 6.7342e-02, 7.3907e-02,
        8.1113e-02, 8.9022e-02, 9.7701e-

In [93]:
test_x = torch.cat([test_x, -test_x]).reshape(-1, 1)
test_x.shape

torch.Size([200, 1])

In [94]:
test_y = (test_x > 0).type(torch.float)
test_y

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
      

In [95]:
model.eval()
with torch.inference_mode():
    y_pred = (model(test_x.to(device)).cpu().sigmoid() > 0.5).type(torch.float)
y_pred

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
      

In [96]:
# 准确率
(test_y == y_pred).type(torch.float).mean().item()

0.8600000143051147

In [97]:
# 获取错误的 x index
error_index = (test_y != y_pred)
error_index

tensor([[ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [ True],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False],
        [False

In [98]:
test_x.shape

torch.Size([200, 1])

In [99]:
# 获取错误的 x
error_x = test_x[error_index.flatten()]
error_x

tensor([[1.0000e-04],
        [1.0975e-04],
        [1.2045e-04],
        [1.3219e-04],
        [1.4508e-04],
        [1.5923e-04],
        [1.7475e-04],
        [1.9179e-04],
        [2.1049e-04],
        [2.3101e-04],
        [2.5354e-04],
        [2.7826e-04],
        [3.0539e-04],
        [3.3516e-04],
        [3.6784e-04],
        [4.0370e-04],
        [4.4306e-04],
        [4.8626e-04],
        [5.3367e-04],
        [5.8570e-04],
        [6.4281e-04],
        [7.0548e-04],
        [7.7426e-04],
        [8.4975e-04],
        [9.3260e-04],
        [1.0235e-03],
        [1.1233e-03],
        [1.2328e-03]])

In [100]:
error_x.numel()

28